In [1]:
# one hot encode runway
# ordinal encode airline
# TODO: Try with avg pb time, ratioH/M, ratio nominal
from traffic.core import Traffic

import pandas as pd
import numpy as np
import pickle

import os.path

from ipyleaflet import Polygon
from ipywidgets import Layout
from ipyleaflet import Map, Marker

import altair as alt

from cartes.osm import Overpass

from sklearn import svm

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix

from numpy import mean
from numpy import std
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
from ground_analysis.build_features import gather_data

/home/service/miniconda3/envs/py39v7/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
%load_ext lab_black

In [4]:
from ground_analysis.utils import within_Xmin
from ground_analysis.extract_flight_info import onehot_runways, assign_twy_occ

In [5]:
from pickle import load, dump

# Load data

In [9]:
dep_arr_df = pd.read_pickle("../data/processed/dep_arr_df10_meansifis.pkl")

In [11]:
def compute_avg_duration(
    dep_df,
    dt="30T",
    time_col="on_runway_time",
    on_col="total_holding_time_minutes",
    col_name="avg_delay",
):
    # Index is on_runway_time because it makes more sense to compute average
    # ground delays with flights having completed their taxi
    avg_delays = (
        dep_df.set_index(time_col)
        .sort_index()[on_col]
        .rolling(dt)  # closed="left" not needed as it is the target variable
        .agg("mean")
        .rename(col_name)
    )
    dep_df = pd.merge_asof(
        dep_df.sort_values(time_col),
        avg_delays,
        left_on=time_col,
        right_index=True,
    )
    return dep_df

/home/service/miniconda3/envs/py39v7/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
Compute avg delays wrt different filters and 

In [360]:
long_enough_taxiing_taxiholdavg_df = dep_arr_df.query(
    'mvt_type=="DEP" & real_dur_minutes > 5'
)
long_enough_taxiing_taxiholdavg_df = compute_avg_duration(
    long_enough_taxiing_taxiholdavg_df,
    on_col="taxi_holding_time_minutes",  # "taxi_holding_time_minutes",
    col_name="long_enough_delay",
)
datachart = long_enough_taxiing_taxiholdavg_df[
    ["first_movement_start", "long_enough_delay"]
]
long_enough_taxiing_taxiholdavg_chart = (
    alt.Chart(datachart)
    .transform_density("long_enough_delay")
    .mark_line(color="orange", strokeDash=[5, 5])
    .encode(x="value:Q", y="density:Q")
)
long_enough_taxiing_taxiholdavg_chart

/home/service/miniconda3/envs/py39v7/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


alt.Chart(...)

In [355]:
long_enough_taxiing_df = dep_arr_df.query('mvt_type=="DEP" & real_dur_minutes > 5')
long_enough_taxiing_df = compute_avg_duration(
    long_enough_taxiing_df,
    on_col="taxi_holding_time_minutes",  # "taxi_holding_time_minutes",
    col_name="long_enough_delay",
)
datachart = long_enough_taxiing_df[["first_movement_start", "long_enough_delay"]]
long_enough_taxiin_chart = (
    alt.Chart(datachart)
    .transform_density("long_enough_delay")
    .mark_line(color="orange")
    .encode(x="value:Q", y="density:Q")
)
long_enough_taxiin_chart

alt.Chart(...)

In [1]:
long_enough_taxiing_df

NameError: name 'long_enough_taxiing_df' is not defined